In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.utils import seed_everything
from swift.tuners import Swift

#微调目录
model_dir1 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/mistral-7b-chat-v2/v7-20231228-133749/checkpoint-17477'
model_dir2 = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/mistral-7b-chat-v2/v9-20240104-151231/checkpoint-250'

model_type1 = ModelType.mistral_7b_chat_v2
# model_type2 = ModelType.zephyr_7b_beta_chat
# model_type = ModelType.zephyr_7b_beta_chat
template_type1 = get_default_template_type(model_type1)
# template_type2 = get_default_template_type(model_type2)

model_mistral, tokenizer_mistral = get_model_tokenizer(model_type1, model_kwargs={'device_map': 'auto'}, model_dir="/home/css/models/Mistral-7B-Instruct-v0.2")

# model_zephyr, tokenizer_zephyr = get_model_tokenizer(model_type2, model_kwargs={'device_map': 'auto'}, model_dir="/home/css/models/zephyr-7b-beta")

model_mistral.generation_config.max_new_tokens = 4096
# model_zephyr.generation_config.max_new_tokens = 4096


#mistral
# model_mistral = Swift.from_pretrained(model_mistral, model_dir1, inference_mode=True)
model_mistral = Swift.from_pretrained(model_mistral, model_dir2, inference_mode=True)

#zephyr
# model_zephyr = Swift.from_pretrained(model_zephyr, model_dir1, inference_mode=True)
# model_zephyr = Swift.from_pretrained(model_zephyr, model_dir2, inference_mode=True)

template_mistral = get_template(template_type1, tokenizer_mistral)
# template_zephyr = get_template(template_type2, tokenizer_zephyr)


/home/qiuyang/miniconda3/envs/tune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-11 13:31:49,944 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-11 13:31:49,946 - modelscope - INFO - Loading ast index from /home/qiuyang/.cache/modelscope/ast_indexer
2024-01-11 13:31:49,987 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 fac241e0e4496f4573eae6f7a58f0aab and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.20s/it]


In [2]:
import json

max_lines = 100
current_line = 0

content_list = []

# 打开JSON文件
with open('/home/qiuyang/workplace/swift/examples/pytorch/llm/my_tuning/data/c4_data.json', 'r') as file:
    # 逐行读取文件内容
    for line in file:
        # 解析JSON数据
        data = json.loads(line)

        length = len(tokenizer_mistral.tokenize(data['content']))
        print(length)
        
        # 在这里可以对每一行的数据进行处理
        content_list.append(data['content'])

        # 更新行数计数器
        current_line += 1
        
        # 检查是否已经达到最大行数
        if current_line >= max_lines:
            break

# content = '''SaharaReporters has learned from reliable sources at the Nigerian Presidency that President Muhammadu Buhari, who has been in the UK since January 19, is receiving \u201cintense treatment\u201d for a renewed flare-up of prostate issues. He had undergone surgical treatment for prostate cancer soon after losing the 2011 presidential election to former President Goodluck Jonathan.\nIn addition to his UK doctors, President Buhari has been under the care of two Nigerian physicians, Dr. Suhayb Sanusi Rafindadi, who is his Chief Personal Physician, and Dr. Ugorji Ogbonna, who had a medical practice for many years in Kano before relocating to the UK. Even though he is from the southeast of Nigeria, Dr. Ogbonna is very close to numerous powerful northern politicians, and one of his sons converted to Islam, according to a source. While Dr. Rafindadi accompanied the president on his medical trips abroad and Dr. Ogbonna often liaise with Mr. Buhari\u2019s physicians in the UK to arrange for his treatments.\nOur sources indicated that Mr. Buhari\u2019s original surgical treatment for prostate cancer had been declared successful. However, when a nagging ear infection forced President Buhari to visit his longtime physician in the UK last year, the prognosis led the president to seek two separate medical opinions, in France and later Germany. Doctors in both countries reportedly told him it was urgent to see his doctors in the UK. Subsequently, Mr. Buhari, who was visiting France for a conference, moved from Paris to London, declaring that he needed a vacation.\nMr. Buhari\u2019s UK doctors advised that he needed to stay put in London for another surgery, but the president bowed to pressure from members of his inner circle and decided to make a premature return to Abuja out of political expediency. However, before President Buhari left London, his doctors there removed polyps from his nostrils in a surgical procedure to ease his breathing.\nIn 2016, President Buhari made trips to the UK in February and June to consult his UK doctors on a variety of health issues, including his ear infection, said our sources.\nSaharaReporters learned that President Buhari currently receives intense treatment for a prostate-related ailment. The treatment at a point severely affected his voice and appetite. He has progressively lost weight and has had to be force-fed on occasion on the orders of his doctors. One source said Mr. Buhari\u2019s treatments had been compounded by his age, which his UK doctors believe to be more than 80 rather than his official \u201cage\u201d of 74 years old.\nInformation gleaned from our sources indicated an attempt to obfuscate the precise nature of Mr. Buhari\u2019s illness, signaling efforts by competing factions around the president to keep the Nigerian public uninformed. According to our sources, four different versions of President Buhari's condition were being circulated. One version, traced to his wife, Aisha Buhari, is that the president has \u201cinternal organ\u201d issues.\nOur sources disclosed that a cabal led by President Buhari's cousin, Mamman Daura, reportedly sent Mrs. Aisha Buhari away from the UK, asking her to make a temporary visit to Saudi Arabia. To justify Aisha Buhari\u2019s exit from London, Mr. Daura reportedly claimed that the president\u2019s condition appeared to worsen each time his wife was around him. He reportedly told Mrs. Buhari that part of her husband's illness was a result of \u201ca spiritual attack,\u201d and asked her to proceed to Saudi Arabia to pray for him. From Saudi Arabia, Mrs. Buhari returned to Nigeria about two weeks ago. One source claimed that she had not returned to London ever since.\nOur sources said Mr. Daura often handled the daily briefing of select Presidency officials about the president's health. According to them, the president\u2019s cousin continues to claim that Mr. Buhari was only exhausted and needed adequate rest before returning to Nigeria ready to take over the mantle of leadership once again.\nAs SaharaReporters revealed last weekend, the doctors treating Mr. Buhari in the UK have told him in clear terms that he ought to shelf any plans to return to serious work and stay back in London for as long as four months to receive a full course of treatment. One source indicated that members of the president\u2019s inner circle were yet to fully embrace the doctors\u2019 recommendation and communicate the information to Nigerians because they fear the loss of political influence.\nOn his part, President Buhari had reportedly told his inner political circle that he was in no hurry to return to Nigeria, adding that he was willing to let the expert advice of his doctors to prevail. However, some of his die-hard associates have not given up plotting to daily to convince him to return to Abuja in defiance of medical advice.\nPresident Buhari reportedly communicates regularly with Acting President Yemi Osinbajo, encouraging him to carry on the task of governing Nigeria while he undertakes necessary treatment needed to keep him alive.'''
# eval_prompt = f"### Instruction: According to the following information：<{content}> Answer the question。Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination，year-month],just country name.\n### Response:\n"






1595
752
4654
202
1938
724
1372
878
1577
91
572
421
567
320
3160
2740
382
4907
485
876
7394
885
4073
722
238
923
474
238
5432
1229
784
1068
295
786
1895
1422
545
550
3541
2114
1198
12813
680
654
547
263
5618
408
1166
5255
513
2505
5532
2961
1110
3317
4341
3184
3574
897
2995
1732
2443
1213
3659
368
1025
878
434
434
1792
1199
3065
6148
1648
631
2265
529
1269
396
485
437
695
1941
1041
2291
1008
824
379
2070
1419
2489
739
4925
2715
5528
1498
745
5759
32889


In [13]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


result_mistral = []

for i in range(0,len(content_list)):
    print(i)
    content = content_list[i]
    eval_prompt = f"### Instruction: According to the following information：<{content}> Answer the question。Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination，year-month],just country name.\n### Response:\n"
    response_mistral, history_mistral = inference(model_mistral, template_mistral, eval_prompt)
    result_mistral.append(response_mistral)

        # 指定本地文件路径
    file_path = 'mistral_list.json'

    # 将数据写入本地文件
    with open(file_path, 'w') as file:
        json.dump(response_mistral, file, indent=2)
    # response_zephyr, history_zephyr = inference(model_zephyr, template_zephyr, eval_prompt)
    # if response_mistral == response_zephyr:
    #     xt = xt + 1

# print(f'Mistral >>> response: {response_mistral}')
# print(f'Zephyr >>> response: {response_zephyr}')

0


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacty of 23.65 GiB of which 4.06 MiB is free. Including non-PyTorch memory, this process has 23.64 GiB memory in use. Of the allocated memory 22.63 GiB is allocated by PyTorch, and 567.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF